In [1]:
import sys
sys.path.insert(0,"../../Ingredient Embeddings")

In [2]:
from ipynb.fs.full.ingredient_embeddings_similarity import exported as Ingredients

Module ingredient_embeddings_similarity.ipynb is loaded


In [4]:
Ingredients.get_most_similar_from_fndds(Ingredients.embed(["Chicken"]))

(0.8468108, 152, 'chicken in whole pieces')

In [5]:
Ingredients.get_ingredient_nutrient_from_fndds("chicken in whole pieces",152)

Main food description                   chicken, back
WWEIA Category description    chicken in whole pieces
Energy (kcal)                                    2.98
Protein (g)                                    0.2573
Carbohydrate (g)                                  0.0
Total Fat (g)                                   0.208
Name: 842, dtype: object

In [6]:
Ingredients.get_most_similar_from_nutrition5k(Ingredients.embed(["Chicken"]))

(1.0, 27)

In [7]:
Ingredients.get_ingredient_nutrient_from_nutrition5k(27)

ingr          chicken
id                 28
cal/g            1.65
fat(g)          0.036
carb(g)           0.0
protein(g)       0.31
Name: 27, dtype: object